# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

import joblib
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

### Workspace & Experiment
The config.json file is downloaded from Azure environment and has to be in the project folder in order for this cell to run.

In [3]:
ws = Workspace.from_config()

experiment_name = 'CapstoneExp'

experiment=Experiment(ws, experiment_name)
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E2J2ADR7E to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-141279
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-141279


### Set Compute cluster

In [4]:


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_target =None
cpu_cluster_name = 'notebook141279'
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.


### Create Dataset

In [5]:


found = False
key = "heart_failure_clinical_records"
description_text = "heart_failure_clinical_records"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

print (found)
if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/DharmeshHub/CapstoneProject/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

print(df.head())
print("*********************************************************************************")
print(df.describe())


False
    age  anaemia  creatinine_phosphokinase  diabetes  ejection_fraction  \
0  75.0        0                       582         0                 20   
1  55.0        0                      7861         0                 38   
2  65.0        0                       146         0                 20   
3  50.0        1                       111         0                 20   
4  65.0        1                       160         1                 20   

   high_blood_pressure  platelets  serum_creatinine  serum_sodium  sex  \
0                    1  265000.00               1.9           130    1   
1                    0  263358.03               1.1           136    1   
2                    0  162000.00               1.3           129    1   
3                    0  210000.00               1.9           137    1   
4                    0  327000.00               2.7           116    0   

   smoking  time  DEATH_EVENT  
0        0     4            1  
1        0     6            1  
2 

In [6]:
ds=dataset
ds

{
  "source": [
    "https://raw.githubusercontent.com/DharmeshHub/CapstoneProject/main/heart_failure_clinical_records_dataset.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "e70f9881-25d0-42df-a1bf-13980d7e5cdf",
    "name": "heart_failure_clinical_records",
    "version": 1,
    "description": "heart_failure_clinical_records",
    "workspace": "Workspace.create(name='quick-starts-ws-141279', subscription_id='610d6e37-4747-4a20-80eb-3aad70a55f43', resource_group='aml-quickstarts-141279')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:

project_folder = './capstone-project'

# TODO: Put your automl settings here
automl_settings = {"experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "enable_early_stopping" : True,
    "n_cross_validations": 2}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [8]:
# TODO: Submit your experiment
automl_run = experiment.submit(automl_config, show_output = True)
automl_run.wait_for_completion()

Running on remote.
No run_configuration provided, running on notebook141279 with default configuration
Running on remote compute: notebook141279
Parent Run ID: AutoML_c6507e5f-cd46-4860-a977-7732236037f7

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputa

{'runId': 'AutoML_c6507e5f-cd46-4860-a977-7732236037f7',
 'target': 'notebook141279',
 'status': 'Completed',
 'startTimeUtc': '2021-03-24T15:53:07.632865Z',
 'endTimeUtc': '2021-03-24T16:34:22.902113Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'notebook141279',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExp","subscription_id":"610d6e37-4747-4a20-80eb-3aad70a55f43","resource_group":"aml-quickstarts-141279","workspace_name":"quick-starts-ws-141279","region":"southcentralus","compute_target":"notebook141279","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":null,"y

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:

RunDetails(automl_run).show()

for child_run in automl_run.get_children():
    print('===================================================')
    print(child_run)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_16,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_17,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_15,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_14,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_12,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_13,
Type: azureml.scriptrun,
Status: Canceled)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_11,
Type: azureml.scriptrun,
Status: Completed)
Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_10,
Type: azureml.scriptrun,
Status: Completed)
Run(

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:

# Get your best run and save the model from that run.

### YOUR CODE HERE ###


print('\n')
print('############################################################')
print('\n')

best_run, fitted_model = automl_run.get_output()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details() ['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n')
print('############################################################')
print('\n')
print("Best run metrics :",best_run.get_metrics())
print('\n')
print('############################################################')
print('\n')
print('Accuracy:', best_run_metrics['accuracy'])
print('\n')
print('############################################################')
print('\n')
print("Best run details :",best_run.get_details())
print('\n')
print('############################################################')
print('\n')
print('Regularization Rate:',parameter_values)
print('\n')
print('############################################################')
print('\n')
print("Best run file names :",best_run.get_file_names())
print('\n')
print('############################################################')
print('\n')


print(best_run)
print(fitted_model)

# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
print("*********************************************************************************")
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())



############################################################


Best Run Id:  AutoML_c6507e5f-cd46-4860-a977-7732236037f7_16


############################################################


Best run metrics : {'AUC_weighted': 0.9001387156927232, 'norm_macro_recall': 0.6198453681030027, 'f1_score_macro': 0.8225792087389807, 'recall_score_micro': 0.852841163310962, 'f1_score_micro': 0.852841163310962, 'log_loss': 0.3885616859169615, 'balanced_accuracy': 0.8099226840515014, 'matthews_correlation': 0.6560385131564732, 'precision_score_weighted': 0.8540951468853424, 'average_precision_score_macro': 0.8835640060269573, 'average_precision_score_weighted': 0.9070004126554221, 'precision_score_micro': 0.852841163310962, 'AUC_macro': 0.9001387156927232, 'f1_score_weighted': 0.8486625205161779, 'AUC_micro': 0.9127678332807831, 'recall_score_weighted': 0.852841163310962, 'recall_score_macro': 0.8099226840515014, 'weighted_accuracy': 0.8861839958679583, 'average_precision_score_micro': 0.913611060

In [11]:
best_run.get_metrics()

{'AUC_weighted': 0.9001387156927232,
 'norm_macro_recall': 0.6198453681030027,
 'f1_score_macro': 0.8225792087389807,
 'recall_score_micro': 0.852841163310962,
 'f1_score_micro': 0.852841163310962,
 'log_loss': 0.3885616859169615,
 'balanced_accuracy': 0.8099226840515014,
 'matthews_correlation': 0.6560385131564732,
 'precision_score_weighted': 0.8540951468853424,
 'average_precision_score_macro': 0.8835640060269573,
 'average_precision_score_weighted': 0.9070004126554221,
 'precision_score_micro': 0.852841163310962,
 'AUC_macro': 0.9001387156927232,
 'f1_score_weighted': 0.8486625205161779,
 'AUC_micro': 0.9127678332807831,
 'recall_score_weighted': 0.852841163310962,
 'recall_score_macro': 0.8099226840515014,
 'weighted_accuracy': 0.8861839958679583,
 'average_precision_score_micro': 0.9136110608127488,
 'precision_score_macro': 0.8478964272442533,
 'accuracy': 0.852841163310962,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_c6507e5f-cd46-4860-a977-7732236037f7_16/

In [27]:
fitted_model = automl_run.get_output()

from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
             print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
        print()

print(fitted_model)





(Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_16,
Type: azureml.scriptrun,
Status: Completed), Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                min_child_weight=1,
                                                                                                min_split_gain=0.8421052631578947,
                  

In [17]:
best_run


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneExp,AutoML_c6507e5f-cd46-4860-a977-7732236037f7_16,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [18]:
best_run.get_file_names()

# Download the yaml file that includes the environment dependencies
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'env.yml')

In [19]:
# Download the model file

best_run.download_file('outputs/model.pkl', 'CapstoneAutoml.pkl')

In [20]:
# Save the best model

best_run.register_model(model_name = "CapstoneAutoml.pkl", model_path = './outputs/')

print(best_run)

Run(Experiment: CapstoneExp,
Id: AutoML_c6507e5f-cd46-4860-a977-7732236037f7_16,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model


model = automl_run.register_model(model_name = 'CapstoneAutoml.pkl')
print(automl_run.model_id)

# https://knowledge.udacity.com/questions/463620

environment = best_run.get_environment()
entry_script='inference/scoring.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)


inference_config = InferenceConfig(entry_script = entry_script, environment = environment)

# Deploying the model via ACI WebService
# https://github.com/MicrosoftDocs/azure-docs/blob/master/articles/machine-learning/how-to-deploy-azure-container-instance.md

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                    memory_gb = 1, 
                                                    auth_enabled= True, 
                                                    enable_app_insights= True)

service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)

CapstoneAutoml.pkl
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-03-24 16:46:27+00:00 Creating Container Registry if not exists.
2021-03-24 16:46:28+00:00 Registering the environment.
2021-03-24 16:46:29+00:00 Use the existing image.
2021-03-24 16:46:29+00:00 Generating deployment configuration.
2021-03-24 16:46:31+00:00 Submitting deployment to compute..
2021-03-24 16:46:36+00:00 Checking the status of deployment aciservice..
2021-03-24 16:53:29+00:00 Checking the status of inference endpoint aciservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [22]:
# Getting the service state
# The scorig URI & the primary authentication key are copied to the endpoint.py file in order to test the deployed service.
# The Swagger URI can be used in Swagger UI: https://petstore.swagger.io/ For more info, please see the relevant part in the README file.

# Authentication is enabled, so I use the get_keys method to retrieve the primary and secondary authentication keys:
primary, secondary = service.get_keys()

print('Service state: ' + service.state)
print('Service scoring URI: ' + service.scoring_uri)
print('Service Swagger URI: ' + service.swagger_uri)
print('Service primary authentication key: ' + primary)

Service state: Healthy
Service scoring URI: http://092c49ec-d4af-498b-86b0-af0c7eaae5aa.southcentralus.azurecontainer.io/score
Service Swagger URI: http://092c49ec-d4af-498b-86b0-af0c7eaae5aa.southcentralus.azurecontainer.io/swagger.json
Service primary authentication key: 9UA2if3bVNVjInr1grTaUKVtTLghO6Ix


TODO: In the cell below, send a request to the web service you deployed to test it.

In [29]:

%run endpoint.py

{"result": [1, 1, 0]}


TODO: In the cell below, print the logs of the web service and delete the service

In [24]:
print(service.get_logs())

2021-03-24T16:53:18,710599700+00:00 - gunicorn/run 
2021-03-24T16:53:18,717907400+00:00 - rsyslog/run 
2021-03-24T16:53:18,716506500+00:00 - iot-server/run 
2021-03-24T16:53:18,759558800+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_2b14f450572e78de640d54eaabed5e4d/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd

In [30]:
service.delete()